In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
def read_network():
    w1 = np.loadtxt('w1.txt')
    w2 = np.loadtxt('w2.txt')
    w3 = np.loadtxt('w3.txt')
    b1 = np.loadtxt('b1.txt')
    b2 = np.loadtxt('b2.txt')
    b3 = np.loadtxt('b3.txt')
    return w1, w2, w3, b1, b2, b3

In [ ]:
w1, w2, w3, b1, b2, b3 = read_network()

In [ ]:
def np2ten(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float64)
    return arg

In [ ]:
x = tf.placeholder(tf.float64, [None, 3], name='x')
y_ = tf.placeholder(tf.float64, [None, 2], name='y_')

w1 = np2ten(w1)
w2 = np2ten(w2)
w3 = np2ten(w3)
b1 = np2ten(b1)
b2 = np2ten(b2)
b3 = np2ten(b3)

l1 = tf.nn.relu(tf.matmul(x, w1) + b1)
l2 = tf.nn.relu(tf.matmul(l1, w2) + b2)
y = tf.matmul(l2, w3) + b3

rows = [tf.gradients(y[:, i], x)[0] for i in range(int(y.shape[1]))]
J2 = tf.stack(rows, axis=1)

sess = tf.InteractiveSession()

In [ ]:
def J0(tau):
    N = 1501

    x_grid = np.linspace(-5, 10, N)
    y_grid = np.linspace(-5, 10, N)
    x_grid, y_grid = np.meshgrid(x_grid, y_grid)

    tau_array = np.ones_like(x_grid.ravel()) * tau

    grid_ravel = np.vstack([x_grid.ravel(), y_grid.ravel(), tau_array]).T
    grid_image = sess.run(y, feed_dict={x: grid_ravel})
    jac = sess.run(J2, feed_dict={x: grid_ravel, y:grid_image})

    jac = np.linalg.det(jac[:, :, :-1])
    jac = jac.reshape((N, N))
    
    ind = np.where((jac[:, :-1] * jac[:, 1:]) < 0)
    J0_x = x_grid[[ind[0], ind[1]]]
    J0_y = y_grid[[ind[0], ind[1]]]
    return np.array([J0_x, J0_y])

In [ ]:
def J_curve0(x, tau, a=1, b=2):
    y = (tau * (1 - tau) * x ** 2 + tau * (b + 1) - 1) / (2 * x * tau)
    return y

In [ ]:
tau_list = [0.01, 0.3]
color_list = 'rk'
fig, ax = plt.subplots(figsize=(12, 6), ncols=2)
for i in range(len(tau_list)):
    J0_points = J0(tau_list[i])
    color = color_list[i]
    ax[1].scatter(J0_points[0], J0_points[1], s=3, c=color, label=r'$\tau = $ {:.2f}'.format(tau_list[i]))
    
    J0_x1 = np.linspace(-10, 0, 1000, endpoint=False)
    J0_x2 = np.linspace(10, 0, 1000, endpoint=False)[::-1]

    J0_y1 = J_curve0(J0_x1, tau=tau_list[i])
    J0_y2 = J_curve0(J0_x2, tau=tau_list[i])
    
    ax[0].plot(J0_x1, J0_y1, color, label=r'$J_0(\tau=$ {:.2f})'.format(tau_list[i]))
    ax[0].plot(J0_x2, J0_y2, color)

    
    ax[i].set_xlim(-5, 10)
    ax[i].set_ylim(-5, 10)
    
    ax[i].tick_params(axis='x', labelsize=16)
    ax[0].tick_params(axis='y', labelsize=16)
    
    ax[1].set_yticklabels(['' for y in ax[1].get_yticks()])
    
    if not i:
        ax[i].set_ylabel('$y$', fontsize=24)
    
    ax[i].set_xlabel('$x$', fontsize=24)
    ax[0].legend(fontsize=16)
    
    ax[0].set_title('the Brusselator Integrator', fontsize=20)
    ax[1].set_title('the Brusselator Network', fontsize=20)
    
plt.tight_layout()

# plt.savefig('ts_j0.pdf', bbox_inches='tight')